<a href="https://colab.research.google.com/github/iam-fern/Research/blob/main/sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.random import normal
import statistics

In [4]:
# numpy.random.normal(loc=0.0, scale=1.0, size=None)
# generate sample of 200 values that follow a normal distribution 
# data = normal(loc=0, scale=1, size=200)

data = pd.DataFrame({"zt1" : normal(0, 1, 50000),
                     "zt2" : normal(0, 1, 50000),
                     "zt3" : normal(0, 1, 5 0000)})
data

,zt1,zt2,zt3
0,0.235963,0.052369,-0.624591
1,-0.387004,0.153847,0.073550
2,0.681346,-0.719791,0.192394
3,-0.428422,0.584047,0.064964
4,-1.713041,1.156098,-1.176319
...,...,...,...
49995,-1.492963,-0.400109,0.156340
49996,0.024184,0.166919,-0.739678
49997,-0.189881,-1.513896,-0.714159
49998,-0.331231,-2.565831,0.164358


In [8]:
n=50
rep=0
table_data_all=pd.DataFrame({"rep":[]})
data_copy=data.copy()
while not data_copy.empty:
    #=========================
    if data_copy.shape[0]>n:
        index=data_copy.sample(n).index
        data_new=data_copy.loc[index,:]
        data_copy=data_copy.drop(index)
    else:
        data_new=data_copy.copy()
        data_copy=pd.DataFrame({"":[]})
    #=========================
    p = 0

    Zt1 = data_new['zt1']
    Zt2 = data_new['zt2']
    Zt3 = data_new['zt3']

    xt1 = (1-(p**2))**(1/2)*Zt1
    xt2 = (1-(p**2))**(1/2)*Zt2
    xt3 = (1-(p**2))**(1/2)*Zt3

    new_data = pd.DataFrame({'Xt1' : data_new['zt1'],
                            'Xt2' : data_new['zt2'],
                            'Xt3' : data_new['zt3'],
                            'e'   : normal(0, xt2**4, n)})


    table_data = pd.DataFrame({'Xt1' : data_new['zt1'],
                            'Xt2' : data_new['zt2'],
                            'Xt3' : data_new['zt3'],
                            'e'   : new_data['e']})
    table_data['Y'] = 4+0.4*table_data['Xt1']+1.5*table_data['Xt2']+3.6*table_data['Xt3']+table_data['e']
    table_data['Y_hat'] = 4+0.4*table_data['Xt1']+1.5*table_data['Xt2']+3.6*table_data['Xt3']
    table_data['e_hat']=table_data['Y']-table_data['Y_hat']
    table_data=table_data.reset_index(drop=True).reset_index()
    table_data["rep"]=rep
    table_data_all=pd.concat([table_data_all,table_data],axis=0,ignore_index=True,sort=False)
    # display(table_data)
    rep+=1
    if rep==100:
      break
table_data_all=table_data_all.set_index(["rep","index"])
table_data_all

Xt1       Xt2       Xt3  ...          Y      Y_hat         e_hat
rep  index                                ...                                    
0.0  0.0   -1.804418  2.007999 -1.150116  ...   7.559760   2.149813  5.409947e+00
     1.0   -0.500343 -1.042903  0.309156  ...   4.620435   3.348470  1.271965e+00
     2.0    0.004097 -0.265473  0.343090  ...   4.841165   4.838553  2.611903e-03
     3.0   -1.327181  1.109103 -1.046049  ...   1.362724   1.367004 -4.279887e-03
     4.0   -0.534790  1.788319 -0.455948  ...  13.050344   4.827149  8.223195e+00
...              ...       ...       ...  ...        ...        ...           ...
99.0 45.0  -0.749606  0.002454 -0.676281  ...   1.269226   1.269226 -5.508283e-11
     46.0   0.532920 -1.694219 -0.168681  ... -18.529537   1.064586 -1.959412e+01
     47.0   0.732524  0.657629 -1.210676  ...   0.685560   0.921018 -2.354574e-01
     48.0  -0.675996  1.083837  0.215292  ...   8.617646   6.130409  2.487236e+00
     49.0   0.961342  2.008491  1.201792  ...  -4.274261  11.723724 -1.599798e+01

[5000 rows x 7 columns]

In [9]:
table_data_all.mean()

Xt1     -0.024894
Xt2      0.018967
Xt3      0.021370
e        0.114828
Y        4.210251
Y_hat    4.095423
e_hat    0.114828
dtype: float64

In [10]:
table_data_all.var()

Xt1        1.023663
Xt2        1.023247
Xt3        1.001631
e         86.262789
Y        101.696965
Y_hat     15.235701
e_hat     86.262789
dtype: float64

In [11]:
# Compute pairwise correlation of columns, excluding NA/null values.
table_data_all.corr()

,Xt1,Xt2,Xt3,e,Y,Y_hat,e_hat
Xt1,1.000000,0.008681,-0.008512,-0.023264,0.016971,0.099201,-0.023264
Xt2,0.008681,1.000000,-0.018051,-0.005025,0.139733,0.372970,-0.005025
Xt3,-0.008512,-0.018051,1.000000,0.007695,0.361304,0.915150,0.007695
e,-0.023264,-0.005025,0.007695,1.000000,0.922056,0.002737,1.000000
Y,0.016971,0.139733,0.361304,0.922056,1.000000,0.389580,0.922056
Y_hat,0.099201,0.372970,0.915150,0.002737,0.389580,1.000000,0.002737
e_hat,-0.023264,-0.005025,0.007695,1.000000,0.922056,0.002737,1.000000
